# Ejercicios ensembling
En este ejercicio vas a realizar prediciones sobre un dataset de ciudadanos indios diabéticos. Se trata de un problema de clasificación en el que intentaremos predecir 1 (diabético) 0 (no diabético).

### 1. Carga las librerias que consideres comunes al notebook

In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold

### 2. Lee los datos de [esta direccion](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv)
Los nombres de columnas son:
```Python
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
```

In [65]:
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

df = pd.read_csv(url, names=['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class'])
df

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   preg    768 non-null    int64  
 1   plas    768 non-null    int64  
 2   pres    768 non-null    int64  
 3   skin    768 non-null    int64  
 4   test    768 non-null    int64  
 5   mass    768 non-null    float64
 6   pedi    768 non-null    float64
 7   age     768 non-null    int64  
 8   class   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [67]:
df.nunique()

preg      17
plas     136
pres      47
skin      51
test     186
mass     248
pedi     517
age       52
class      2
dtype: int64

### 3. Bagging
Para este apartado tendrás que crear un ensemble utilizando la técnica de bagging ([BaggingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html)), mediante la cual combinarás 100 [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html). Recuerda utilizar también [cross validation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) con 10 kfolds.

**Para este apartado y siguientes, no hace falta que dividas en train/test**, por hacerlo más sencillo. Simplemente divide tus datos en features y target.

Establece una semilla

In [68]:
# Separar características (X) y etiquetas (y)
X = df.drop('class', axis=1)
y = df['class']

In [69]:
# Establecemos la semilla para reproducibilidad
RANDOM_SEED = 42

In [70]:
kfold = KFold(n_splits=10, shuffle=True, random_state=RANDOM_SEED)

In [71]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

estimator = DecisionTreeClassifier(max_depth=3,random_state=42)

bag_clf = BaggingClassifier(
    estimator = estimator,
    n_estimators=100, # Cantidad de árboles
    max_samples=100, # Muestras utilizadas en boostrapping
    bootstrap=True, # Usamos boostrapping
    # max_features = 3 # Features que utiliza en el boostrapping. Cuanto más bajo, mejor generalizará y menos overfitting
    random_state=42)


bag_clf.fit(X, y)
y_pred = bag_clf.predict(X)
accuracy_score(y, y_pred)

0.78125

In [72]:
# Validación cruzada
kfold = 10
scores = cross_val_score(bag_clf, X, y, cv = kfold, scoring ='accuracy')

# Mostrar resultados
bag_clf_mean_accuracy = np.mean(scores)
print(f"Precisión promedio del modelo Bagging: {bag_clf_mean_accuracy:.3f}")

Precisión promedio del modelo Bagging: 0.771


### 4. Random Forest
En este caso entrena un [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) con 100 árboles y un `max_features` de 3. También con validación cruzada

In [73]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=100,
                                 max_leaf_nodes=3,
                                 random_state=42)
rnd_clf.fit(X, y)

y_pred_rf = rnd_clf.predict(X)
# np.sum(y_test == y_pred_rf) / len(y_test) 
accuracy_score(y, y_pred_rf)

0.7643229166666666

In [74]:
# Validación cruzada
scores = cross_val_score(rnd_clf, X, y, cv = kfold, scoring ='accuracy')

# Mostrar resultados
rnd_clf_mean_accuracy = np.mean(scores)
print(f"Precisión promedio del modelo Bagging: {rnd_clf_mean_accuracy:.3f}")

Precisión promedio del modelo Bagging: 0.758


### 5. GradientBoosting
Implementa un [GradientBoostingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) con 100 estimadores

In [75]:
from sklearn.ensemble import GradientBoostingClassifier

gbct = GradientBoostingClassifier(max_depth=2,
                                 n_estimators=100,
                                 learning_rate=1.0,
                                 random_state=42)
gbct.fit(X, y)


y_pred_gbct = gbct.predict(X)
accuracy_score(y, y_pred_gbct)

0.9947916666666666

In [76]:
# Validación cruzada para Gradient Boosting
gb_scores = cross_val_score(gbct, X, y, cv=kfold, scoring='accuracy')

In [77]:
# Promedio de precisión
gb_mean_accuracy = np.mean(gb_scores)
print(f"Precisión promedio del modelo Gradient Boosting: {gb_mean_accuracy:.3f}")

Precisión promedio del modelo Gradient Boosting: 0.705


### 6. XGBoost
Para este apartado utiliza un [XGBoostClassifier](https://docs.getml.com/latest/api/getml.predictors.XGBoostClassifier.html) con 100 estimadores. XGBoost no forma parte de la suite de modelos de sklearn, por lo que tendrás que instalarlo con pip install

In [78]:
import xgboost

xgb_clas = xgboost.XGBClassifier(
    n_estimators=100,
    random_state=RANDOM_SEED
)

xgb_clas.fit(X, y)
y_pred = xgb_clas.predict(X)
accuracy_score(y, y_pred)

1.0

In [79]:
# Validación cruzada para Gradient Boosting
xgb_scores = cross_val_score(xgb_clas, X, y, cv=kfold, scoring='accuracy')

In [80]:
# Promedio de precisión
xgb_mean_accuracy = np.mean(xgb_scores)
print(f"Precisión promedio del modelo Gradient Boosting: {xgb_mean_accuracy:.3f}")

Precisión promedio del modelo Gradient Boosting: 0.743


### 7. Primeros resultados
Crea un dataframe con los resultados y sus algoritmos, ordenándolos de mayor a menor

In [81]:
# Resultados de los modelos
resultados = {
    "Algoritmo": ["Bagging", "Random Forest", "Gradient Boosting", "XGBoost"],
    "Precisión Promedio": [bag_clf_mean_accuracy, rnd_clf_mean_accuracy, gb_mean_accuracy, xgb_mean_accuracy]  # Incluye los valores calculados previamente
}

# Crear un DataFrame
resultados_df = pd.DataFrame(resultados)

# Ordenar los resultados de mayor a menor
resultados_df = resultados_df.sort_values(by="Precisión Promedio", ascending=False)

# Mostrar el DataFrame ordenado
print(resultados_df)


           Algoritmo  Precisión Promedio
0            Bagging            0.770848
1      Random Forest            0.757826
3            XGBoost            0.743472
2  Gradient Boosting            0.704511


### 9. Hiperparametrización
Vuelve a entrenar los modelos de nuevo, pero esta vez dividiendo el conjunto de datos en train/test y utilizando un gridsearch para encontrar los mejores hiperparámetros.

In [82]:
# Dividir el conjunto de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [83]:
bag_clf = BaggingClassifier(
    estimator = DecisionTreeClassifier(random_state=42),
    random_state=42)

from sklearn.model_selection import GridSearchCV

# Define el rango de hiperparámetros para el Grid Search
param_grid = {
    "n_estimators": [50, 100, 150, 200],
    "max_samples": [2, 5, 10, 20],
    #"bootstrap": [True, False]
}

# Configura el Grid Search con Cross-Validation
grid_search = GridSearchCV(
    estimator=bag_clf,
    param_grid=param_grid,
    scoring="accuracy",  # Métrica a optimizar
    cv=5,  # Número de divisiones de Cross-Validation
    verbose=2,
    n_jobs=-1  # Usar todos los núcleos disponibles
)

# Ajusta el modelo con los datos
grid_search.fit(X_train, y_train)

# Imprime los mejores hiperparámetros y el mejor puntaje
print("Mejores hiperparámetros:", grid_search.best_params_)
print("Mejor puntuación (accuracy):", grid_search.best_score_)

# EXtraer el mejor modelo
best_model = grid_search.best_estimator_

# Realiza predicciones con el modelo optimizado
y_pred = best_model.predict(X_test)

# Calcular métricas de rendimiento (opcional)
from sklearn.metrics import accuracy_score
accuracy_bagging = accuracy_score(y_test, y_pred)
print("Accuracy Bagging en test:", accuracy_bagging)




Fitting 5 folds for each of 16 candidates, totalling 80 fits
Mejores hiperparámetros: {'max_samples': 20, 'n_estimators': 150}
Mejor puntuación (accuracy): 0.7622151139544182
Accuracy Bagging en test: 0.7467532467532467


In [84]:
rnd_clf = RandomForestClassifier(random_state=42)
rnd_clf.fit(X, y)

from sklearn.model_selection import GridSearchCV

# Define el rango de hiperparámetros para el Grid Search
param_grid = {
    "n_estimators": [50, 100, 150, 200],
    "max_leaf_nodes": [2, 5, 10, 20],
}

# Configura el Grid Search con Cross-Validation
grid_search = GridSearchCV(
    estimator=bag_clf,
    param_grid=param_grid,
    scoring="accuracy",  # Métrica a optimizar
    cv=5,  # Número de divisiones de Cross-Validation
    verbose=2,
    n_jobs=-1  # Usar todos los núcleos disponibles
)

# Ajusta el modelo con los datos
grid_search.fit(X_train, y_train)

# Imprime los mejores hiperparámetros y el mejor puntaje
print("Mejores hiperparámetros:", grid_search.best_params_)
print("Mejor puntuación (accuracy):", grid_search.best_score_)

# EXtraer el mejor modelo
best_model = grid_search.best_estimator_

# Realiza predicciones con el modelo optimizado
y_pred = best_model.predict(X_test)

# Calcular métricas de rendimiento (opcional)
from sklearn.metrics import accuracy_score
accuracy_Random_Forest = accuracy_score(y_test, y_pred)
print("Accuracy Bagging en test:", accuracy_Random_Forest)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


ValueError: Invalid parameter 'max_leaf_nodes' for estimator BaggingClassifier(estimator=DecisionTreeClassifier(random_state=42),
                  random_state=42). Valid parameters are: ['bootstrap', 'bootstrap_features', 'estimator', 'max_features', 'max_samples', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'].

In [ ]:
gbct = GradientBoostingClassifier(learning_rate=1.0,
                                 random_state=42)
gbct.fit(X, y)

from sklearn.model_selection import GridSearchCV

# Define el rango de hiperparámetros para el Grid Search
param_grid = {
    "max_depth": [50, 100, 150, 200],
    "n_estimators": [2, 5, 10, 20],
}

# Configura el Grid Search con Cross-Validation
grid_search = GridSearchCV(
    estimator=bag_clf,
    param_grid=param_grid,
    scoring="accuracy",  # Métrica a optimizar
    cv=5,  # Número de divisiones de Cross-Validation
    verbose=2,
    n_jobs=-1  # Usar todos los núcleos disponibles
)

# Ajusta el modelo con los datos
grid_search.fit(X_train, y_train)

# Imprime los mejores hiperparámetros y el mejor puntaje
print("Mejores hiperparámetros:", grid_search.best_params_)
print("Mejor puntuación (accuracy):", grid_search.best_score_)

# EXtraer el mejor modelo
best_model = grid_search.best_estimator_

# Realiza predicciones con el modelo optimizado
y_pred = best_model.predict(X_test)

# Calcular métricas de rendimiento (opcional)
from sklearn.metrics import accuracy_score
accuracy_Gradient_Boosting = accuracy_score(y_test, y_pred)
print("Accuracy Bagging en test:", accuracy_Gradient_Boosting)

### 10. Conclusiones finales